In [ ]:
import copy
import re
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

import cobra

cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"

ModelDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/model/'
TpmDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/multi_omics/transcriptomic/'

# Functions

## make_irreversible

In [ ]:
def make_irreversible(cobra_model):
    """
    Convert a COBRApy model to be irreversible.
    :param cobra_model: COBRApy model written in SBML3
    :return: irreversible COBRApy model
    """
    irr_cobra_model = copy.deepcopy(cobra_model)
    reactions = list(irr_cobra_model.reactions)
    for r in reactions:
        if r.lower_bound < 0 < r.upper_bound:
            # If the reaction can go both ways, split it into two irreversible reactions
            new_rxn_fwd = cobra.Reaction(r.id + '_fwd')
            new_rxn_rev = cobra.Reaction(r.id + '_rev')
            new_rxn_fwd.gene_reaction_rule = r.gene_reaction_rule
            new_rxn_rev.gene_reaction_rule = r.gene_reaction_rule
            for met, coeff in r.metabolites.items():
                new_rxn_fwd.add_metabolites({met: coeff})
                new_rxn_rev.add_metabolites({met: -coeff})
            new_rxn_fwd.lower_bound = 0
            new_rxn_fwd.upper_bound = r.upper_bound
            new_rxn_rev.lower_bound = 0
            new_rxn_rev.upper_bound = -r.lower_bound
            irr_cobra_model.add_reactions([new_rxn_fwd, new_rxn_rev])
            r.remove_from_model()
        elif r.lower_bound < 0 and r.upper_bound == 0:            
            # If the reaction only goes in the reverse direction, flip the reaction direction
            r.lower_bound, r.upper_bound = r.upper_bound, -r.lower_bound
            for met, coeff in r.metabolites.items():
                r.add_metabolites({met: -2 * coeff})

    return irr_cobra_model

## get_matrix

In [ ]:
def get_matrix(cobra_model):
    """
    Create Numpy stoichiometric matrix from COBRApy model
    :param cobra_model: COBRApy model written in SBML3
    :return: Numpy stoichiometric matrix, columns with reactions and rows with metabolites
    """

    met_remove_list = []
    for metabolite in cobra_model.metabolites:
        if len(metabolite.reactions) == 0:
            met_remove_list.append(metabolite)
    cobra_model.remove_metabolites(met_remove_list)

    matrix = np.zeros((len(list(cobra_model.metabolites)), len(list(cobra_model.reactions))))

    for j, rxn in enumerate(list(cobra_model.reactions)):
        for met, coeff in rxn.metabolites.items():
            i = list(cobra_model.metabolites).index(met)
            matrix[i, j] = coeff

    return matrix

## map_expression_to_reaction

In [ ]:
def map_expression_to_single_rule(rule, operator, expression):
    """
    Map expression for single gene-reaction-rule. The Rules that have genes bound with parenthesis should not be used
    :param rule: gene-reactions-rule
    :param operator: either one of 'and' or 'or'
    :param expression: Pandas dataframe first column with gene ID and second column with expression value
    :return: expression value (float) mapped to the input rule
    """
    single_rule = re.split(r'\s', rule)
    expr_values = []
    genes = [k for k in single_rule if k != operator]

    for gene in genes:
        try:
            gene = float(gene)
            expr_values.append(gene)
        except:
            if gene in list(expression.gene):
                gene_index = list(expression.gene).index(gene)
                expr_values.append(float(expression.iloc[gene_index, 1]))
            else:
                expr_values.append(float(0))
    
    if operator == 'and':
        single_rule_value = min(expr_values)
    else: # operator == 'or'
        single_rule_value = max(expr_values)

    return single_rule_value

def map_expression_to_reaction(model, expression):
    """
    Map expression values to all reactions in the model.
    The reactions with missing gene-reaction-rule have Nan.
    The genes with missing expression value will be replaced to zero.
    :param model: COBRApy metabolic model written and read as SBML3 (.xml)
    :param expression: Pandas dataframe first column with gene ID and second column with expression value
    :return: Pandas dataframe first column with reaction ID and second column with mapped expression value
    """
    rxn_expression_dict = {}

    for r in model.reactions:
        rule = r.gene_reaction_rule
        num_sub_rules = rule.count('(')
        if rule == "":
            rxn_expression_dict[r.id] = np.nan

        elif num_sub_rules == 0:

            if 'and' in rule:
                final_rule_value = map_expression_to_single_rule(rule, 'and', expression)

            else:  # 'or' in rule:
                final_rule_value = map_expression_to_single_rule(rule, 'or', expression)

            rxn_expression_dict[r.id] = final_rule_value

        else:
            while num_sub_rules != 0:
                open_index = [i for i, character in enumerate(rule) if character == '(']
                close_index = [i for i, character in enumerate(rule) if character == ')']
                combined_index = open_index + close_index
                combined_index.sort()
                lowest_level = {}
                for i in range(len(combined_index)):

                    if combined_index[i] in open_index and combined_index[i + 1] in close_index:
                        sub_rule = rule[combined_index[i] + 1:combined_index[i + 1]]

                        if 'and' in sub_rule:
                            sub_rule_value = map_expression_to_single_rule(sub_rule, 'and', expression)
                        else:  # 'or' in sub_rule:
                            sub_rule_value = map_expression_to_single_rule(sub_rule, 'or', expression)

                        lowest_level[rule[combined_index[i]:combined_index[i + 1] + 1]] = sub_rule_value

                for key in list(lowest_level.keys()):
                    rule = rule.replace(key, str(lowest_level[key]))

                num_sub_rules = rule.count('(')

            if 'and' in rule:
                final_rule_value = map_expression_to_single_rule(rule, 'and', expression)
            else:  # 'or' in final_rule:
                final_rule_value = map_expression_to_single_rule(rule, 'or', expression)

            rxn_expression_dict[r.id] = final_rule_value

    rxn_expression_df = pd.DataFrame.from_dict(rxn_expression_dict, orient='index', columns=['value'])
    return rxn_expression_df

# Creating Gurobi model: test with sMMO model

In [ ]:
md = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_OB3b_sMMO.xml')
md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
md.reactions.NGAM_c0.bounds = (25.3,25.3)

## Generate met-rxn matrix

In [ ]:
md_matrix = get_matrix(md)

## Gurobi empty model to add vars/contrs/obj

In [ ]:
gp_md = gp.Model(name = 'sMMO_gurobi_model')
gp_md

## Create/add variables to model

In [ ]:
n_rxns = len(md.reactions)
rxn_id = [rxn.id for rxn in md.reactions]
rxn_lb = [rxn.lower_bound for rxn in md.reactions]
rxn_ub = [rxn.upper_bound for rxn in md.reactions]

gp_vars = gp_md.addVars(n_rxns, lb = rxn_lb, ub = rxn_ub, name = rxn_id)
gp_md.update()
gp_md

#### This also works
# for reaction in list(md.reactions):
#     if reaction.id == 'bio1_biomass':
#         gp_md.addVar(lb=reaction.lower_bound, ub=reaction.upper_bound, obj = 0.0, name = reaction.id)
#     else:
#         gp_md.addVar(lb=reaction.lower_bound, ub=reaction.upper_bound, obj = 0.0, name = reaction.id)
# gp_md.update()
# gp_md


## Create/add constraints to model

In [ ]:
for n_row, row in enumerate(md_matrix):
    nonzero_indices = np.nonzero(row)[0]
    constraints_list = [md_matrix[n_row, rxn_index] * gp_vars[rxn_index] for rxn_index in nonzero_indices]
    # constraints_list = []
    # for rxn_index in nonzero_indices:
    #     coeff = md_matrix[n_row, rxn_index]
    #     variable = gurobi_variables[rxn_index]
    #     constraints_list.append(coeff * variable)
    gp_md.addConstr(sum(constraints_list) == 0)

gp_md.update()
gp_md

## Set objective

In [ ]:
print(gp_md.getVarByName('bio1_biomass').obj)
gp_obj = gp.LinExpr(1*gp_md.getVarByName('bio1_biomass'))
gp_md.setObjective(expr = gp_obj, sense = GRB.MAXIMIZE)
gp_md.update()
print(gp_md.getVarByName('bio1_biomass').obj)

## Optimize model

In [ ]:
gp_md_sol = gp_md.optimize()

In [ ]:
gp_md.ObjVal

# GIMME application using gurobipy

In [ ]:
def gimme(md, obj_id, obj_fraction, expression, threshold):
    '''
    md: cobra model
    
    obj_id: objective function id in model
    
    obj_fraction: objective fraction to be optimized
    
    expression: pandas dataframe with two columns of gene_id and corresponding value, e.g., TPM or FPKM
    
    threshold: gene expression value threshold
    
    [TEST LOG]
    
    March 26, 2024: function was tested and returned same result from the Matlab GIMME, where inputs were
        
        md = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_OB3b_sMMO.xml')
        obj_id = 'bio1_bioamss'
        obj_fraction = 0.9
        expression = pd.read_csv(TpmDir + 'TPM_con1_NoCu_NoCe.csv', sep=',', index_col=None)
        threshold = 300
    
    
    '''
    ### Copy input model
    gimme_md = copy.deepcopy(md)
    
    
    ### Initial optimization and set lower bound with input fraction
    md_pfba = cobra.flux_analysis.pfba(md)
    md.reactions.get_by_id(obj_id).bounds = (obj_fraction * md_pfba.fluxes.loc[obj_id], 1000)
    
    ### create irreversible model, stoichiometic matrix, map expression to reactions
    irr_md = make_irreversible(md)
    irr_md_matrix = get_matrix(irr_md)
    irr_tpm_mapped_df = map_expression_to_reaction(irr_md, expression)
    
    
    ### Generate gurobi model to solve for GIMME

    # Model frame
    gp_gimme_md = gp.Model(name = 'Gurobipy_model_GIMME')
    
    # Add variables
    n_rxns = len(irr_md.reactions)
    rxn_id = [rxn.id for rxn in irr_md.reactions]
    rxn_lb = [rxn.lower_bound for rxn in irr_md.reactions]
    rxn_ub = [rxn.upper_bound for rxn in irr_md.reactions]
    gp_gimme_vars = gp_gimme_md.addVars(n_rxns, lb = rxn_lb, ub = rxn_ub, name = rxn_id)
    
    # Add constraints
    for n_row, row in enumerate(irr_md_matrix):
        nonzero_indices = np.nonzero(row)[0]
        constraints_list = [irr_md_matrix[n_row, rxn_index] * gp_gimme_vars[rxn_index] for rxn_index in nonzero_indices]
        gp_gimme_md.addConstr(sum(constraints_list) == 0)
    
    # Add objective coefficient
    c_array = np.zeros(len(irr_md.reactions))
    for rxn_ind, rxn in enumerate(irr_md.reactions):
        if rxn.id in irr_tpm_mapped_df.index and irr_tpm_mapped_df.loc[rxn.id, 'value'] < threshold:
            c_array[rxn_ind] = threshold - irr_tpm_mapped_df.loc[rxn.id, 'value']
    
    # Add objective
    objectives_list = []
    for rxn_ind in range(len(irr_md.reactions)):
        objectives_list.append(c_array[rxn_ind] * gp_gimme_vars[rxn_ind])
    gp_gimme_obj = gp.LinExpr(sum(objectives_list))
    gp_gimme_md.setObjective(expr = gp_gimme_obj, sense = GRB.MINIMIZE)
    
    gp_gimme_md.update()
    
    # Solve GIMME
    gp_gimme_md_sol = gp_gimme_md.optimize()
    
    ### Find reactions that can be deleted in the model
    irr_reaction_activity_dict = {}

    for i in gp_gimme_vars:

        if gp_gimme_vars[i].VarName in irr_tpm_mapped_df.index and irr_tpm_mapped_df.loc[gp_gimme_vars[i].VarName, 'value'] > threshold:
            irr_reaction_activity_dict[gp_gimme_vars[i].VarName] = 1

        elif gp_gimme_vars[i].VarName not in irr_tpm_mapped_df.index:
            irr_reaction_activity_dict[gp_gimme_vars[i].VarName] = 2

        elif gp_gimme_vars[i].X > 0:
            irr_reaction_activity_dict[gp_gimme_vars[i].VarName] = 3

        else:
            irr_reaction_activity_dict[gp_gimme_vars[i].VarName] = 0

    
    reaction_activity_array = np.zeros(len(md.reactions))

    for rxn_index, rxn in enumerate(md.reactions):

        rxn_reversibility = rxn.lower_bound * rxn.upper_bound

        if rxn_reversibility >= 0:
            reaction_activity_array[rxn_index] = irr_reaction_activity_dict[rxn.id]

        else:
            rxn_activity = max(irr_reaction_activity_dict[rxn.id + '_fwd'],
                               irr_reaction_activity_dict[rxn.id + '_rev'])
            reaction_activity_array[rxn_index] = rxn_activity
    
    ### Remove non-essential reactions and return modified cobra model
    rxn_list_to_remove = []
    for rxn_index, activity in enumerate(reaction_activity_array):
        if activity == 0:
            rxn_list_to_remove.append(gimme_md.reactions[rxn_index])
    
    gimme_md.remove_reactions(rxn_list_to_remove, remove_orphans=True)
    
    ### Return GIMME model and inconsistency score
    return gimme_md, gp_gimme_md.ObjVal
    

In [ ]:
smmo = ModelDir + "GEM_ADVE_OB3b_sMMO.xml"
pmmo_ra = ModelDir + "GEM_ADVE_OB3b_pMMO_RA.xml"
pmmo_ut = ModelDir + "GEM_ADVE_OB3b_pMMO_UT.xml"
pmmo_dc = ModelDir + "GEM_ADVE_OB3b_pMMO_DC.xml"

avg_tpm = [27, 34, 25, 45]

c_assim = ['rxn07847_c0','rxn07848_c0','rxn37236_c0','rxn02480_c0','rxn02431_c0','rxn07849_c0','rxn00371_c0',
           'rxn00690_c0','rxn01211_c0','rxn00907_c0','rxn12217_c0', 'rxn00692_c0']
phb_syn = ['rxn00992_c0','rxn00290_c0','phb_polymerase_c0', 'phb_depolymerase_c0']
ser_cyc = ['rxn00424_c0','rxn01011_c0','rxn01013_c0','rxn01102_c0',
           'rxn01106_c0','rxn00459_c0','rxn00251_c0','rxn00248_c0', 'rxn00934_c0', 'rxn00331_c0']
tca_cyc = ['rxn00256_c0','rxn00974_c0','rxn01388_c0','rxn00198_c0','rxn08094_c0']
emc_cyc = ['rxn00178_c0','rxn01451_c0','rxn01453_c0','rxn02345_c0',
           'rxn02167_c0','rxn16150_c0','rxn16825_c0','rxn16151_c0','rxn25269_c0',
           'rxn03440_c0','rxn00682_c0','rxn01355_c0','rxn01996_c0','rxn00602_c0', 'rxn00285_c0', 'rxn09272_c0','rxn00799_c0']
pyr_met = ['rxn00147_c0','rxn00148_c0','rxn00151_c0','rxn00154_c0','rxn00161_c0']
n_assim = ['rxn00182_c0', 'rxn00184_c0', 'rxn00085_c0', 'rxn00187_c0', 'rxn00568_c0', 'rxn00569_c0', 'rxn00572_c0', 'rxn10120_c0']

central_rxns = c_assim + phb_syn + ser_cyc + tca_cyc + emc_cyc + pyr_met + n_assim

In [ ]:
expression = pd.read_csv(TpmDir + 'TPM_con1_NoCu_NoCe.csv', sep=',', index_col=None)
tpm_threshold = 27

md = cobra.io.read_sbml_model(smmo)
md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
md.reactions.NGAM_c0.bounds = (25.3,25.3)

gimme_md, inconsistency = gimme(md, 'bio1_biomass', 0.9, expression, tpm_threshold)

# nn = no Cu + no Ce
cobra.io.write_sbml_model(gimme_md, ModelDir + 'tpm_integrated_model/' + 'gimme_nn.xml')

In [ ]:
expression = pd.read_csv(TpmDir + 'TPM_con3_NoCu_YesCe.csv', sep=',', index_col=None)
tpm_threshold = 25

md = cobra.io.read_sbml_model(smmo)
md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
md.reactions.NGAM_c0.bounds = (25.3,25.3)

gimme_md, inconsistency = gimme(md, 'bio1_biomass', 0.9, expression, tpm_threshold)

# nn = no Cu + yes Ce
cobra.io.write_sbml_model(gimme_md, ModelDir + 'tpm_integrated_model/' + 'gimme_ny.xml')

In [ ]:
pmmo_md = [pmmo_ra, pmmo_ut, pmmo_dc]
gimme_name = ['gimme_yn_ra.xml', 'gimme_yn_ut.xml', 'gimme_yn_dc.xml']
expression = pd.read_csv(TpmDir + 'TPM_con2_YesCu_NoCe.csv', sep=',', index_col=None)
tpm_threshold = 34

for md_file, name in zip(pmmo_md, gimme_name):
    md = cobra.io.read_sbml_model(md_file)
    md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
    md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
    md.reactions.NGAM_c0.bounds = (25.3,25.3)
    
    gimme_md, inconsistency = gimme(md, 'bio1_biomass', 0.9, expression, tpm_threshold)

    cobra.io.write_sbml_model(gimme_md, ModelDir + 'tpm_integrated_model/' + name)


In [ ]:
pmmo_md = [pmmo_ra, pmmo_ut, pmmo_dc]
gimme_name = ['gimme_yy_ra.xml', 'gimme_yy_ut.xml', 'gimme_yy_dc.xml']
expression = pd.read_csv(TpmDir + 'TPM_con4_YesCu_YesCe.csv', sep=',', index_col=None)
tpm_threshold = 45

for md_file, name in zip(pmmo_md, gimme_name):
    md = cobra.io.read_sbml_model(md_file)
    md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
    md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
    md.reactions.NGAM_c0.bounds = (25.3,25.3)
    
    gimme_md, inconsistency = gimme(md, 'bio1_biomass', 0.9, expression, tpm_threshold)

    cobra.io.write_sbml_model(gimme_md, ModelDir + 'tpm_integrated_model/' + name)


# OptGene model preparation

In [ ]:
smmo = ModelDir + "GEM_ADVE_OB3b_sMMO.xml"
pmmo_ra = ModelDir + "GEM_ADVE_OB3b_pMMO_RA.xml"
pmmo_ut = ModelDir + "GEM_ADVE_OB3b_pMMO_UT.xml"
pmmo_dc = ModelDir + "GEM_ADVE_OB3b_pMMO_DC.xml"

gimme_nn = ModelDir + "tpm_integrated_model/gimme_nn.xml"
gimme_ny = ModelDir + "tpm_integrated_model/gimme_ny.xml"
gimme_yn_ra = ModelDir + "tpm_integrated_model/gimme_yn_ra.xml"
gimme_yn_ut = ModelDir + "tpm_integrated_model/gimme_yn_ut.xml"
gimme_yn_dc = ModelDir + "tpm_integrated_model/gimme_yn_dc.xml"
gimme_yy_ra = ModelDir + "tpm_integrated_model/gimme_yy_ra.xml"
gimme_yy_ut = ModelDir + "tpm_integrated_model/gimme_yy_ut.xml"
gimme_yy_dc = ModelDir + "tpm_integrated_model/gimme_yy_dc.xml"

model_files = [
    smmo, pmmo_ra, pmmo_ut, pmmo_dc,
    gimme_nn, gimme_ny, 
    gimme_yn_ra, gimme_yn_ut, gimme_yn_dc, 
    gimme_yy_ra, gimme_yy_ut, gimme_yy_dc
]
model_names = [
    'gk_smmo', 'gk_pmmo_ra', 'gk_pmmo_ut', 'gk_pmmo_dc',
    'gk_gimme_nn', 'gk_gimme_ny', 
    'gk_gimme_yn_ra', 'gk_gimme_yn_ut', 'gk_gimme_yn_dc', 
    'gk_gimme_yy_ra', 'gk_gimme_yy_ut', 'gk_gimme_yy_dc'
]

for file, name in zip(model_files, model_names):
    
    md = cobra.io.read_sbml_model(file)
    
    md.reactions.EX_cpd00007_e0.bounds = (-21.2,0)
    md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
    md.reactions.NGAM_c0.bounds = (25.3,25.3)

    putrescine_export = cobra.Reaction("putrescine_export")
    putrescine_export.name = 'putrescine_export'
    putrescine_export.lower_bound = 0
    putrescine_export.upper_bound = 1000
    putrescine_export.add_metabolites({
        md.metabolites.get_by_id('cpd00118_c0'): -1.0
    })
    # putrescine_export.gene_reaction_rule = ''
    md.add_reactions([putrescine_export])

    cobra.io.write_sbml_model(md, ModelDir + 'optgene_model/' + name + '.xml')
